# Homework 3 - Chapter 8

- Due Date: Monday, February 9th no later than 11:59 p.m.
- Partner Information: Learning to collaborate with others is an important skill and
one that employers value.  This assignment requires you to partner with a classmate and submit 
one solution for the two of you.
- Submission Instructions: Upload your solution, entitled 
**YourFirstName-YourLastName-PartnerFirstName-PartnerLastName-Homework3.ipynb** to the 
Canvas Homework 3 Dropbox.  Note: only one person needs to submit a solution. If you and your partner 
both submit a solution, the submission that will be graded is the one from the partner whose last name comes 
alphabetically first.
- Deadline Reminder: Once the submission deadline passes, Canvas will no longer accept your submission and you will no longer be able to earn credit. 
Thus, if you are not able to fully complete the assignment, submit whatever you have before the deadline so that partial credit can be earned.

## Starting Code

In [2]:
from datascience import *
import numpy as np

Download the file [nfl_2024.csv](https://www.cs.montana.edu/paxton/classes/fall-2025/intro-ds/homeworks/hw3/nfl_2024.csv)
into the same directory as this Jupyter notebook.

In [3]:
# Place the csv file in the same directory as your solution
season = Table().read_table("nfl_2024.csv")
season

Week,Day,Date,Time,Winner,Loser,Winner Points,Loser Points,Winner Yards,Loser Yards
1,Thu,9/5/2024,8:20PM,Kansas City Chiefs,Baltimore Ravens,27,20,353,452
1,Fri,9/6/2024,8:15PM,Philadelphia Eagles,Green Bay Packers,34,29,410,414
1,Sun,9/8/2024,1:00PM,Pittsburgh Steelers,Atlanta Falcons,18,10,270,226
1,Sun,9/8/2024,1:00PM,Buffalo Bills,Arizona Cardinals,34,28,352,270
1,Sun,9/8/2024,1:00PM,New Orleans Saints,Carolina Panthers,47,10,379,193
1,Sun,9/8/2024,1:00PM,Chicago Bears,Tennessee Titans,24,17,148,244
1,Sun,9/8/2024,1:00PM,New England Patriots,Cincinnati Bengals,16,10,290,224
1,Sun,9/8/2024,1:00PM,Houston Texans,Indianapolis Colts,29,27,417,303
1,Sun,9/8/2024,1:00PM,Miami Dolphins,Jacksonville Jaguars,20,17,400,267
1,Sun,9/8/2024,1:00PM,Minnesota Vikings,New York Giants,28,6,312,240


The Super Bowl is almost upon us. Working with data collected from the 2025 regular season, you will utilize
your Data Science skills to make predictions about future games, even if you have no football expertise.

## Question 1 - 3 Points

Knowing that *defense wins championships*, you want to determine which games the winning team held their opponent below the average (mean) points scored by losing teams across the season. 

Create a function that returns **True** if the points scored by the losing team (**Loser Points**) is below the mean of that column 
and **False** otherwise. Apply this function to the table, adding a new column entitled **Loser Below Average?** with these values. 
Display the first 7 entries in the augmented table.

In [4]:
totalLoserPoints = season.column("Loser Points").sum()
numOfLosers = season.num_rows
meanLoserPoints = totalLoserPoints/numOfLosers

def belowAverage(loserPoints):
    if (loserPoints  < meanLoserPoints):
        return True
    else:
        return False
    
question1 = season.with_column("Loser Below Average?", season.apply(belowAverage, "Loser Points"))
question1.show(7)

Week,Day,Date,Time,Winner,Loser,Winner Points,Loser Points,Winner Yards,Loser Yards,Loser Below Average?
1,Thu,9/5/2024,8:20PM,Kansas City Chiefs,Baltimore Ravens,27,20,353,452,False
1,Fri,9/6/2024,8:15PM,Philadelphia Eagles,Green Bay Packers,34,29,410,414,False
1,Sun,9/8/2024,1:00PM,Pittsburgh Steelers,Atlanta Falcons,18,10,270,226,True
1,Sun,9/8/2024,1:00PM,Buffalo Bills,Arizona Cardinals,34,28,352,270,False
1,Sun,9/8/2024,1:00PM,New Orleans Saints,Carolina Panthers,47,10,379,193,True
1,Sun,9/8/2024,1:00PM,Chicago Bears,Tennessee Titans,24,17,148,244,True
1,Sun,9/8/2024,1:00PM,New England Patriots,Cincinnati Bengals,16,10,290,224,True


## Question 2 - 2 Points

Let's use the column that was just created to make a prediction for this year's Super Bowl. Using Python, create a table with two columns: (1) the name of a football team - call this *Team* and (2) the number of times that when that team won, the losing team's point total was below average - call this *Big Wins*.  Display the first 7 entries in this 2-column table in descending order by *Big Wins*.  **Hint** - If done correctly, the sixth entry in the table will have 7 Big Wins.

In [5]:
question2 = question1.group("Winner",sum)
question2 = question2.relabel(["Winner", "Loser Below Average? sum"], ["Team", "Big Wins"])
question2 = question2.select("Team", "Big Wins")
question2 = question2.sort("Big Wins", descending=True)
question2.show(7)

Team,Big Wins
Philadelphia Eagles,10
Kansas City Chiefs,9
Detroit Lions,8
Los Angeles Chargers,8
Minnesota Vikings,8
Baltimore Ravens,7
Buffalo Bills,7


Assume that that top two teams in the table you produced for Question 2 will play in the Super Bowl.

## Question 3a - 2 Points

Point differential is another important statistic when it comes to determining success in the Super Bowl. Create a pivot table where the columns are the two teams that Question 2 predicts will play in the Super Bowl and the rows are the values of the **Loser
Below Average** column. Only consider games that the two Super Bowl contenders won.  Each entry in the table should be the mean point differential - calculated by subtracting the points scored by the losing team from the points scored by the winning team. Display the pivot table.  The mean point differentials should be formatted to display 2 digits
to the right of the decimal.

In [23]:
# Place answer here.

top2 = question2.column("Team")[:2]
all_team = question1.where("Winner", are.contained_in(top2))
all_team = all_team.with_column(
    "Point Differential",
        all_team.column("Winner Points") - all_team.column("Loser Points")
)
all_team = all_team.pivot("Winner", "Loser Below Average?", values="Point Differential", collect=np.mean)
all_team = all_team.set_format([1, 2], NumberFormatter(2))
all_team

Loser Below Average?,Kansas City Chiefs,Philadelphia Eagles
False,5.62,11.14
True,8.11,15.30


## Question 3b - 1 Point

Using information from the above pivot table, explain carefully which of the two teams
you would predict to win the Super Bowl.

**Your answer:** 
Philadelphia would most likely win the Super Bowl. When the loser is either below average or not, Philadelphia Eagles scores higher with the point differential compared to Kansas City Chiefs. 

## Question 4 - 2 Points

Discover something insightful with the provided data set. 
Explain what you have done and why it is insightful.

In [44]:
question1 = question1.with_column(
    "Point Differential",
    question1.column("Winner Points") - question1.column("Loser Points")
)

new_table = question1.group("Winner", np.mean).select("Winner", "Point Differential mean")
high_table = new_table.sort("Point Differential mean", descending=True)
high_table.show(5)
low_table = new_table.sort("Point Differential mean", descending=False)
low_table.show(5)

Winner,Point Differential mean
Denver Broncos,18
New Orleans Saints,17.8
Tampa Bay Buccaneers,16.8
Detroit Lions,15.4667
Baltimore Ravens,14.8462


Winner,Point Differential mean
Indianapolis Colts,3.75
Cleveland Browns,5
Carolina Panthers,6
Las Vegas Raiders,6.75
Kansas City Chiefs,6.94118


**Explanation -**

This analysis shows which teams win by the biggest margins and which teams win by the smallest margins. Teams at the top of the list (high average point differential) crush their opponents when they win, while teams at the bottom barely edge out victories. This shows which teams always most likely would crush their opponent vs those who are always in a close tight games with their opponents.